## SQL 에이전트 구축하기 

**LangChain 에이전트**를 사용하여 **SQL 데이터베이스에 대한 질문에 답변할 수 있는 에이전트**를 만드는 방법을 배웁니다.

---

### 에이전트의 전체 동작 흐름

1. 데이터베이스에서 사용 가능한 **테이블과 스키마 정보**를 가져옵니다.
2. 질문과 관련된 **적절한 테이블을 선택**합니다.
3. 선택된 테이블의 **스키마(schema)** 를 조회합니다.
4. 질문과 스키마 정보를 기반으로 **SQL 쿼리를 생성**합니다.
5. **LLM을 이용해 쿼리를 검증**하고, 흔한 오류가 있는지 확인합니다.
6. 쿼리를 실행하고 **결과를 반환**합니다.
7. 데이터베이스 엔진이 반환한 오류를 분석해 **쿼리를 수정**하고 재시도합니다.
8. 최종적으로 **결과를 바탕으로 자연어 응답을 생성**합니다.

---

### 보안 주의 사항

SQL 데이터베이스용 Q&A 시스템을 구축할 때는
**모델이 자동으로 SQL 쿼리를 생성하고 실행**하므로 본질적인 위험이 있습니다.
따라서 다음을 반드시 지켜야 합니다.

* 데이터베이스 연결 권한을 **에이전트가 필요한 범위로 최소화(scope 최소화)**
* 권한을 제한하면 위험이 완전히 사라지지는 않지만, **피해 범위를 줄일 수 있습니다.**

---

다음 개념들을 다룹니다:

* **SQL 데이터베이스에서 데이터를 읽어오는 도구 (Tools for reading from SQL databases)**
* **LangChain 에이전트 (LangChain agents)**
* **Human-in-the-loop 프로세스**
  → 사람이 중간에 개입하여 쿼리나 결과를 검증하는 안전 장치


**요약:**  
LangChain을 이용해 **SQL 질의응답(Q&A)** 시스템을 구축하고,  
모델이 생성한 SQL을 실행·검증·보정하며,  
결과를 기반으로 **자연스러운 대답을 생성하는 에이전트**를 만드는 것입니다.

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

### **샘플 데이터**

이 예제에서는 **SQLite** 연결을 사용하여 **Chinook 데이터베이스**를 활용합니다.  
Chinook 데이터베이스는 **디지털 미디어 스토어**를 나타내는 샘플 데이터베이스입니다.

---

##  **Chinook 데이터베이스 설치**

[설치 가이드](https://database.guide/2-sample-databases-sqlite/)를 따라 `Chinook.db`를 현재 노트북과 같은 디렉터리에 생성합니다.  

또는, 아래 명령어를 사용하여 커맨드 라인에서 데이터베이스를 다운로드하고 빌드할 수 있습니다:

```bash
curl -s https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql | sqlite3 Chinook.db
```

### **설명:**  
1. `curl` 명령어를 사용하여 Chinook 데이터베이스 SQL 스크립트를 다운로드합니다.  
2. `sqlite3`를 사용해 SQL 스크립트를 실행하고 `Chinook.db` 파일을 생성합니다.

---
이 데이터베이스는 **SQLAlchemy 기반의 `SQLDatabase` 클래스**를 사용하여 상호작용할 수 있습니다.

In [ ]:
# SQLite3 설치 in Colab
# !sudo apt-get update
# !sudo apt-get install sqlite3

In [ ]:
# Chinook db 설치 in Colab
# !curl -s https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql | sqlite3 Chinook.db

In [ ]:
# SQLite 데이터베이스 파일 'Chinook.db'에 연결합니다.
# 사용 중인 데이터베이스의 SQL 방언(SQL Dialect)을 출력합니다.
# 데이터베이스에서 사용할 수 있는 테이블 이름들을 출력합니다.
# Artist 테이블의 상위 10개 레코드를 조회합니다

## **데이터베이스 상호작용 도구 추가하기**

langchain_community 패키지에서 제공하는 SQLDatabase 래퍼(wrapper) 를 사용하면 SQL 데이터베이스와 쉽게 상호작용할 수 있습니다.

이 래퍼는 SQL 쿼리 실행 및 결과 조회(fetch) 를 위한 간단하고 일관된 인터페이스를 제공합니다.

### **Agent 생성**
create_agent를 사용하면 최소한의 코드로 ReAct 에이전트를 만들 수 있습니다.  
이 에이전트는 사용자의 요청을 해석하고, 그에 맞는 SQL 명령(SQL command) 을 생성하며, 등록된 도구(tools) 를 이용해 명령을 실행합니다. 

### 동작 과정
- 사용자의 질의(request)를 해석합니다.  
- SQL 쿼리를 생성합니다.  
- 등록된 도구(tools)가 해당 SQL 명령을 실행합니다.  
- 만약 SQL 실행 중 오류가 발생하면, 오류 메시지(error message) 를 모델로 다시 전달합니다.  
- 모델은 원래 질의 + 오류 메시지를 함께 참고하여 새로운 SQL 쿼리를 생성합니다.  
- 이 과정은 모델이 올바른 SQL 명령을 생성할 때까지 또는 지정된 반복 횟수에 도달할 때까지 계속됩니다.

In [ ]:
# 에이전트를 스트리밍 모드로 실행
    # 마지막 메시지(모델 응답)를 보기 좋게 출력

### **인간 검토(Human-in-the-loop) 기능 구현하기**

에이전트가 생성한 SQL 쿼리를 실행하기 전에 의도하지 않은 동작이나 비효율적인 쿼리가 포함되어 있는지 검토하는 것이 좋습니다.  
LangChain의 에이전트는 이러한 인간 개입형 검토 프로세스(Human-in-the-loop) 를 간단히 추가할 수 있도록 내장 미들웨어(Middleware) 를 제공합니다.
이 기능을 사용하면, 에이전트가 특정 도구(예: sql_db_query)를 호출할 때 일시 정지(pause) 하여 사람이 쿼리를 직접 검토하고 승인할 수 있습니다.  
즉, 아래 설정을 통해 에이전트가 sql_db_query 도구를 호출할 때마다 사람의 확인 절차를 거친 후에만 실행되도록 구성할 수 있습니다.

우리는 에이전트에 **체크포인터(checkpointer)** 를 추가하여 에이전트의 실행을 **일시 중지(pause)** 했다가 **다시 재개(resume)** 할 수 있도록 만들었습니다.

이제 에이전트를 실행하면, `sql_db_query` 도구를 실행하기 전에 **인간 검토를 위해 일시 정지**하게 됩니다.

In [ ]:
# 인터럽트에는 `action_requests` 및 `review_configs`가 포함된 전체 HITL(Human-In-The-Loop) 요청이 들어 있습니다.

----------
동일한 실행(run)을 반복하되, 간단한 **예/아니오(yes/no) 승인 단계**를 추가해보겠습니다.

In [ ]:
# 승인 결정으로 실행 재개

In [ ]:
        # 결정(decisions)은 검토 중인 각 작업(action)마다 하나씩 리스트 형태로 제공합니다.
        # 결정의 순서는 `__interrupt__` 요청에 나열된 작업 순서와 일치해야 합니다.
                    # 해당 작업을 거절한 이유에 대한 설명 메시지

에이전트는 필요한 정보를 얻을 때까지 **여러 번의 쿼리**를 실행한다는 점에 주목하세요:  

1. 사용 가능한 테이블 목록을 나열합니다.  
2. 세 개의 테이블에 대한 스키마를 검색합니다.  
3. 여러 테이블을 **JOIN 연산**을 통해 쿼리합니다.  

에이전트는 마지막 쿼리의 결과를 사용해 **원래 질문에 대한 답변을 생성**할 수 있습니다.

에이전트는 이와 유사하게 **정성적 질문(qualitative questions)** 도 처리할 수 있습니다.